In [52]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from pyIEEM.data.data import get_hospitalisation_incidence
from pyIEEM.models.utils import initialize_model

# settings
country = 'SWE'
nrows=3
ncols=4
start_calibration = '2020-02-01'
end_calibration = '2020-03-22'

# get data
data = get_hospitalisation_incidence(country)

# setup model
age_classes = pd.IntervalIndex.from_tuples([(0,5),(5,10),(10,15),(15,20),(20,25),(25,30),(30,35),(35,40),(40,45),(45,50),(50,55),(55,60),(60,65),(65,70),(70,75),(75,80),(80,120)], closed='left')
model = initialize_model(country, age_classes, True, simulation_start)

In [83]:
5*[(0,10),]

[(0, 10), (0, 10), (0, 10), (0, 10), (0, 10)]

In [53]:
# helper function to adjust initial condition
def update_initial_condition(infected, initial_condition, N_other, N_work):
    assert len(infected) == initial_condition.shape[1]
    # compute number of contacts per age group per spatial patch (N x G)
    N = np.sum(N_other,axis=0) + np.sum(N_work, axis=0)
    # pre-allocate output
    output = np.zeros(initial_condition.shape, dtype=float)
    # loop over spatial patches
    for j,inf in enumerate(infected):
        # determine index of age group to drop infected in
        i = np.random.choice(len(N[:, 0]), p=N[:, 0]/sum(N[:, 0]))
        # assign to output
        output[i,j] = inf
    return output

In [80]:
def SSE(theta, data, model, start_date, end_date):
    
    # change initial condition
    model.initial_states['E'] = update_initial_condition(theta, initial_condition, model.parameters['N']['other'], model.parameters['N']['work'])

    # simulate model
    out = model.sim([start_date, end_date])
    
    # slice right data out of dataframe
    data = data.loc[slice(start_calibration, end_calibration)]

    # interpolate model output to dates in data
    out = out.interp({'date': data.index.get_level_values('date').unique().values}, method="linear")
    
    # extract hospitalisations
    out = out.Hin.sum(dim='age_class').sel(spatial_unit=data.index.get_level_values('spatial_unit').unique().values).values
    
    # compute SSE
    data = data.values.reshape(out.shape)
    SSE = np.sum((out-data)**2)
    
    return SSE

In [81]:
infected = np.zeros(21)
infected[0] = 1

SSE(infected, data, model, start_calibration, end_calibration)

[[  0.       0.       0.       0.       0.       0.       0.       0.
    0.       0.       0.       0.       0.       0.       0.       0.
    0.       0.       0.       0.    ]
 [  0.       0.       0.       0.       0.       0.       0.       0.
    0.       0.       0.       0.       0.       6.303    0.597    0.
    0.       0.       0.       1.368 ]
 [  0.       0.7905   1.5      0.       0.       1.0785   0.       0.
    0.       0.       1.2105   0.       1.2315  13.599    0.6435   0.5085
    0.       0.       0.       2.799 ]
 [  0.       2.589    2.8815   1.2255   1.8165   5.6625   1.335    0.
    2.247    2.4      6.423    3.498    1.929   51.051    1.5165   1.119
    0.6      0.5865   0.951    8.5275]
 [  0.5895   8.451    4.8945   3.6495   1.6395  17.169    1.5105   1.0215
    3.216    4.5     18.5235   6.297    8.0895 110.3895   6.2985   0.8895
    1.0395   1.5525   5.904   14.0775]
 [  0.762    8.019    7.026    2.235    3.912   21.393    3.0555   0.
    7.362    5.727  

39636.631887232164